In [1]:
import pandas as pd
import gc
PATH = '/home/kai/data/kaggle/homecredit/'
pos = pd.read_csv(PATH + 'POS_CASH_balance.csv')
df  = pd.read_pickle(PATH + 'inter/poscash2curr.pkl')

from sklearn.linear_model import LinearRegression

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join(gp_col + [col])})
    return group

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index()

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index()

def linear(df, gp_col, x_col_name, y_col_name):
    r = []
    for i, x in df.groupby(gp_col):
        lg = LinearRegression()
        lg1 = LinearRegression()
        
        lg.fit(x[[x_col_name]], x[[y_col_name]])
        x_max = max(x[x_col_name])
        x_min = min(x[x_col_name])
        lg1.fit((x[[x_col_name]]-x_max)/(x_max-x_min+1), x[[y_col_name]])
        r.append([i, lg.coef_[0][0], lg1.coef_[0][0]])
    tmpdf = pd.DataFrame(r, columns=[gp_col, x_col_name+'_lg_'+y_col_name, x_col_name+'_normallg_'+y_col_name])
    return tmpdf

def feature_in_time_window(df, gp_col, col, func, agg_fun=None, n=None, time_col=None):
    tmp = None
    _df = df.copy()
    if n is not None:
        _df = _df[_df[time_col] >= n]
    if agg_fun is None:
        tmp = func(_df, gp_col, col)
    else:
        tmp = func(_df, gp_col, col, agg_fun)
    columns = [str(abs(n))+'_'+x for x in set(tmp.columns) - set(gp_col)]
    tmp.columns = columns
    del _df
    gc.collect()
    return tmp

def area_under_curve(df, gp_col, x, y):
    gp = df.copy()
    gp_max = gp.groupby(gp_col)[[x]].max().reset_index().rename(columns={x:'max'})
    gp_min = gp.groupby(gp_col)[[x]].min().reset_index().rename(columns={x:'min'})
    gp = gp.merge(gp_max)
    gp = gp.merge(gp_min)
    gp['normal_x'] = (gp[x]-gp['max']) / (gp['max']-gp['min']+1)
    
    group = gp.groupby(gp_col)
    gp['tmp'] = (group[y].shift(-1)+gp[y]) * (group[x].shift(-1)-gp[x]) / 2
    gp['tmp_normal'] = (group[y].shift(-1)+gp[y]) * (group['normal_x'].shift(-1)-gp['normal_x']) / 2
    return gp.groupby(gp_col).agg({'tmp':'sum', 'tmp_normal':'sum'}).reset_index().rename(columns={'tmp':x+'_area_'+y, 'tmp_normal':x+'_normalarea_'+y})

'done'

'done'

In [2]:
#  1. how many records, positive sk_dpd, sk_dpd_def a prev id has---> agg(records, skdpd, sk_dpd_def) of curr id

    
positive_dpd = positive_count(pos, ['SK_ID_CURR', 'SK_ID_PREV'],'SK_DPD')  
positive_def = positive_count(pos, ['SK_ID_CURR', 'SK_ID_PREV'],'SK_DPD_DEF')
# How many records a prev id has
prev_count = pos[['SK_ID_CURR','SK_ID_PREV','SK_DPD']].groupby(['SK_ID_PREV','SK_ID_CURR'])[['SK_DPD']].count().reset_index().rename(index=str, columns={'SK_DPD':'count_prev'})

# group1 = numerical(positive_dpd, 'SK_ID_CURR','positivecount_SK_ID_CURR_SK_ID_PREV_SK_DPD', ['sum','max','mean'])
# group2 = numerical(positive_def, 'SK_ID_CURR','positivecount_SK_ID_CURR_SK_ID_PREV_SK_DPD_DEF', ['sum','max','mean'])
group3 = numerical(prev_count, 'SK_ID_CURR','count_prev', ['sum','max','mean'])
for group in [group3]:
    df = df.merge(group, on = 'SK_ID_CURR', how = 'left')
    print(df.shape)

(356255, 119)


In [3]:
# 2. pct of sk_dpd and sk_dpd def in a prev id --> agg into curr
pct_count = positive_dpd.merge(prev_count.drop('SK_ID_CURR', axis = 1), on = 'SK_ID_PREV', how = 'left')
pct_count = pct_count.merge(positive_def.drop('SK_ID_CURR', axis = 1), on = 'SK_ID_PREV', how = 'left')
pct_count = pct_count.rename(index = str, columns = {'positivecount_SK_ID_CURR_SK_ID_PREV_SK_DPD': 'dpd','positivecount_SK_ID_CURR_SK_ID_PREV_SK_DPD_DEF': 'def'})

pct_count[ratio_name('def','count_prev')] = ratio(pct_count,'def','count_prev')
pct_count[ratio_name('dpd','count_prev')] = ratio(pct_count,'dpd','count_prev')

group1 = numerical(pct_count, 'SK_ID_CURR','dpd_divide_count_prev',['max','sum','mean'])
group2 = numerical(pct_count, 'SK_ID_CURR','def_divide_count_prev',['max','sum','mean'])

for group in [group1, group2]:
    df = df.merge(group, on = 'SK_ID_CURR', how = 'left')
    print(df.shape)

(356255, 122)
(356255, 125)


In [4]:
# Count how many UNIQUE prev id per curr id
x = pos[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR'])[['SK_ID_PREV']].nunique().reset_index().\
rename(index = str, columns = {'SK_ID_PREV': 'unique_prev'})
df = df.merge(x, how = 'left', on = 'SK_ID_CURR')

#  3**.pct of completed/ num of prev id, for each curr id
# df[ratio_name('poscash_onehot_NAME_CONTRACT_STATUS_Completed','unique_prev')] = ratio(df, 'poscash_onehot_NAME_CONTRACT_STATUS_Completed','unique_prev')

In [5]:
df[ratio_name('poscash_onehot_NAME_CONTRACT_STATUS_Completed','unique_prev')] = ratio(df, 'poscash_onehot_NAME_CONTRACT_STATUS_Completed','unique_prev')

In [6]:
# SK_DPD
print('shift for dpd')
group1 = pos[['SK_ID_CURR','SK_ID_PREV','SK_DPD']].groupby(['SK_ID_CURR','SK_ID_PREV'])[['SK_DPD']].shift(-1)
pos['shifted'] = group1
# In shift(-1), sk_dpd minus shifted is positive meaning its worsening.
pos['diff'] = pos['SK_DPD'] - pos['shifted']
tmp = numerical(pos, ['SK_ID_CURR','SK_ID_PREV'],'diff',['max','sum'])
# group onto curr
group1 = numerical(tmp, 'SK_ID_CURR', 'diff_max',['mean','max','std'])
group2 = numerical(tmp, 'SK_ID_CURR', 'diff_sum',['mean','max','std'])

for group in [group1, group2]:
    df = df.merge(group, on = 'SK_ID_CURR', how = 'left')
    print(df.shape)
    
# count how many negative instances
dpd_neg = pos.loc[pos['diff'] < 0]

x = count(dpd_neg, 'SK_ID_CURR', 'diff')
df = df.merge(x, on = 'SK_ID_CURR', how = 'left')
print(df.shape)


print('shift for def')   
# SK_DPD_DEF
group1 = pos[['SK_ID_CURR','SK_ID_PREV','SK_DPD_DEF']].groupby(['SK_ID_CURR','SK_ID_PREV'])[['SK_DPD_DEF']].shift(-1)
pos['shifted_def'] = group1
# In shift(-1), sk_dpd minus shifted is positive meaning its worsening.
pos['diff_def'] = pos['SK_DPD'] - pos['shifted_def']
tmp = numerical(pos, ['SK_ID_CURR','SK_ID_PREV'],'diff_def',['max','sum'])
# group onto curr
group1 = numerical(tmp, 'SK_ID_CURR', 'diff_def_max',['mean','max','std'])
group2 = numerical(tmp, 'SK_ID_CURR', 'diff_def_sum',['mean','max','std'])
for group in [group1, group2]:
    df = df.merge(group, on = 'SK_ID_CURR', how = 'left')
    print(df.shape)

    
# count how many negative instances
def_neg = pos.loc[pos['diff_def'] < 0]
x = count(def_neg, 'SK_ID_CURR', 'diff')
df = df.merge(x, on = 'SK_ID_CURR', how = 'left')
print(df.shape)
del x,tmp, group1, group2
gc.collect()

shift for dpd
(356255, 130)
(356255, 133)
(356255, 134)
shift for def
(356255, 137)
(356255, 140)
(356255, 141)


231

In [7]:
# 6. unique count for count_installment --> agg(sum, mean, min, max, std) for curr id

x = pos[['SK_ID_CURR','SK_ID_PREV','CNT_INSTALMENT']].groupby(['SK_ID_CURR','SK_ID_PREV'])[['CNT_INSTALMENT']].nunique().reset_index().\
rename(index = str, columns = {'CNT_INSTALMENT': 'unique_cnt_install'})
# agg(unique for curr)
df = df.merge(numerical(x, 'SK_ID_CURR','unique_cnt_install',['sum','mean','std','min','max']), on = 'SK_ID_CURR', how = 'left')
print(df.shape)


(356255, 146)


In [8]:
# 8. last record for prev id: num of future, and get future installment/count_installment --> curr id
x = pos.groupby(['SK_ID_PREV']).first().reset_index()
x['possible_future_install'] = x['CNT_INSTALMENT_FUTURE']/x['CNT_INSTALMENT']
# Pct of future install/install

df = df.merge(numerical(x,'SK_ID_CURR', 'possible_future_install',['max','std','sum']), on = 'SK_ID_CURR', how = 'left')
# Last record 
df = df.merge(numerical(x, 'SK_ID_CURR', 'CNT_INSTALMENT_FUTURE',['max']), on = 'SK_ID_CURR', how = 'left')
print(df.shape)

(356255, 150)


In [9]:

df.to_pickle(PATH + 'test/pos/concat_pos.pkl')
# df.to_pickle(PATH + 'test/pos/poscash2curr_60.pkl')
# 'done'

In [10]:
# df1 = pd.read_pickle(PATH + 'inter/poscash2curr.pkl')
# df2 = pd.read_pickle(PATH + 'test/pos/poscash2curr_24.pkl')
# df3 = pd.read_pickle(PATH + 'test/pos/poscash2curr_60.pkl')

# df = pd.concat([df1,df2.drop('SK_ID_CURR',axis = 1),df3.drop('SK_ID_CURR', axis = 1)], axis =1)
# df.to_pickle(PATH + 'test/pos/ori_36_pos.pkl')